# Importação das Bibliotecas

In [12]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
from composicao_histograma_boxplot import composicao_histograma_boxplot
from composicao_histograma_boxplot import calcular_estatisticas

# Leitura e Análise da Base

In [7]:
# Leitura da planilha 
DADOS = r"C:\Users\rodri\OneDrive\Sincronização\Documentos\GitHub\hypothesis_test_covid-19\dados\teste_hipoteses.xlsx"

df_virus_enfraquecido = pd.read_excel(DADOS, sheet_name="virus_enfraquecido")
df_rna_mensageiro = pd.read_excel(DADOS, sheet_name="rna_mensageiro")

# Visualizar as primeiras linhas
print("Amostra Vírus Enfraquecido")
print("--------------------------")
print(df_virus_enfraquecido.head())
print("="*40)
print("Amostra RNA Mensageiro")
print("--------------------------")
print(df_rna_mensageiro.head())

Amostra Vírus Enfraquecido
--------------------------
   Nº Amostra Sintomas  Ind. Graves
0           1    Leves            0
1           2   Graves            1
2           3    Leves            0
3           4    Leves            0
4           5    Leves            0
Amostra RNA Mensageiro
--------------------------
   Nº Amostra Sintomas  Ind. Graves
0           1    Leves            0
1           2    Leves            0
2           3   Graves            1
3           4   Graves            1
4           5   Graves            1


In [8]:
# Conferir os tipos de dados das duas amostras
print("Amostra Vírus Enfraquecido")
print("--------------------------")
print(df_virus_enfraquecido.info())
print("="*40)
print("Amostra RNA Mensageiro")
print("--------------------------")
print(df_rna_mensageiro.info())

Amostra Vírus Enfraquecido
--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nº Amostra   38 non-null     int64 
 1   Sintomas     38 non-null     object
 2   Ind. Graves  38 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.0+ KB
None
Amostra RNA Mensageiro
--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nº Amostra   50 non-null     int64 
 1   Sintomas     50 non-null     object
 2   Ind. Graves  50 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ KB
None


# Identificação do Problema

Com todos os esforços para combater o COVID-19, surgiram diversas iniciativas para desenvolver uma vacina eficaz para reduzir o impacto dos sintomas. Um laboratório em suas pesquisas chegou a duasestratégias viáveis:

1. Vírus enfraquecido
2. RNA mensageiro

Para decidir por qual estratégia seguir, realizou um estudo clínico com 87 pessoas distribuídas aleatoriamente entre as duas estratégias. Com os resultados obtidos, como podemos responder: 
- Existe diferença na estratégia em relação à proporção de sintomas graves?

In [9]:
# Cálculo das proporções
n_virus_enfraquecido = len(df_virus_enfraquecido)
prop_graves_virus = (df_virus_enfraquecido["Ind. Graves"].mean() * 100).round(1)
prop_leves_virus = 100 - prop_graves_virus

n_rna_mensageiro = len(df_rna_mensageiro)
prop_graves_rna = (df_rna_mensageiro["Ind. Graves"].mean() * 100).round(1)
prop_leves_rna = 100 - prop_graves_rna

# Criando um DataFrame organizado
df_resumo = pd.DataFrame({
    "Grupo": ["Vírus Enfraquecido", "RNA Mensageiro"],
    "Tamanho Amostra": [n_virus_enfraquecido, n_rna_mensageiro],
    "Proporção Graves": [prop_graves_virus, prop_graves_rna],
    "Proporção Leves": [prop_leves_virus, prop_leves_rna]
})

# Exibir o DataFrame
df_resumo

,Grupo,Tamanho Amostra,Proporção Graves,Proporção Leves
0,Vírus Enfraquecido,38,15.8,84.2
1,RNA Mensageiro,50,28.0,72.0


Para responder a pergunta precisamos saber se a **proporção** de pessoas com sintomas graves tendo sido vacinadas com o **vírus enfraquecido (VE)** é **estatisticamente diferente** da **proporção** de pessoas com sintomas graves que foram vacinadas com o **RNA mensageiro (RNA)**.

Definimos então as seguintes hipóteses nula e alternativa:
- H0: A proporção sintomas graves VE é igual a proporção sintomas graves RNA, ou:
pVE = pRNA
- H1: A proporção sintomas graves VE é diferente da proporção sintomas graves
RNA, ou: pVE ≠ pRNA

In [22]:
# Proporções e tamanhos das amostras
p1 = df_resumo.loc[0, "Proporção Graves"] / 100  # Proporção de casos graves no grupo 1 (vírus enfraquecido)
p2 = df_resumo.loc[1, "Proporção Graves"] / 100  # Proporção de casos graves no grupo 2 (RNA mensageiro)
n1 = df_resumo.loc[0, "Tamanho Amostra"]  # Tamanho da amostra do grupo 1
n2 = df_resumo.loc[1, "Tamanho Amostra"]  # Tamanho da amostra do grupo 2

# Proporção combinada (ponderada)
p_comb = ((p1 * n1) + (p2 * n2)) / (n1 + n2)  # Calcula a proporção combinada (média ponderada)

# Erro padrão da diferença de proporções
SE = np.sqrt(p_comb * (1 - p_comb) * (1/n1 + 1/n2))  # Cálculo do erro padrão

# Cálculo da estatística Z
z = (p1 - p2) / SE  # Estatística Z para a diferença entre as proporções

# Cálculo do valor-p (teste bicaudal)
p_valor = (2 * (1 - norm.cdf(abs(z))) * 100).round(0)  # Valor-p para o teste bicaudal

# Exibir resultados
print("Para esta comparação utilizaremos o Teste-Z para 2 populações")
print(f"Estatística Z: {z:.3f}")  # Exibe a estatística Z calculada
print("="*40)
print(" ")
print("Após realizar o Teste-Z, podemos calcular o p-valor")
print(f"Valor-p: {p_valor:.0f}%")  # Exibe o valor-p calculado

Para esta comparação utilizaremos o Teste-Z para 2 populações
Estatística Z: -1.353
 
Após realizar o Teste-Z, podemos calcular o p-valor
Valor-p: 18%


Como o **p-valor de 18% é maior que o nível de significância de 5%**, podemos concluir que **não existem evidências estatísticas suficientes contra H0**, ou seja, não rejeitamos H0.

Relembrando as hipóteses definidas:
- H0: A proporção sintomas graves VE é igual a proporção sintomas graves RNA
- H1: A proporção sintomas graves VE é diferente da proporção sintomas graves RNA

E como não rejeitamos H0, podemos dizer que não existem evidências estatísticas de que a proporção de pessoas com sintomas graves não é igual nos 2 tipos de vacinas.